In [1]:
import torch
import numpy
from torch import nn
import torchvision
from torchvision import transforms, datasets
from torch import optim

In [2]:
import torch.nn.functional as F

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,),(0.5,)),
                              ])

In [4]:
trainset= datasets.MNIST('~/.pytorch/MNIST_data//',download=True,train=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=100,shuffle=True)


# for images,label in trainloader:
#     print(images.shape)
#     break
    
# Output:
# torch.Size([100, 1, 28, 28])

In [5]:
testset= datasets.MNIST('~/.pytorch/MNIST_data//',download=True,train=True,transform=transform)
testloader = torch.utils.data.DataLoader(trainset,batch_size=100,shuffle=True)

In [6]:
class CNNModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        #Conv 1
        self.cnn1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.relu1 = nn.ReLU()
        
        #maxpool1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        #Conv 2
        self.cnn2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.relu2 = nn.ReLU()
        
        #Maxpool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # 7 = (28/2)/2
        self.fc1 = nn.Linear(32*7*7,10)
        
    def forward(self,x):
        # x is of size (64,1,28,28)
        # reshape to (64,784)
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)

        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.shape[0],-1)
        
        out = self.fc1(out)
        return out
        

In [7]:
model = CNNModel()

In [8]:
print(model)

CNNModel(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
)


In [9]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [10]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr=learning_rate)

In [11]:
epochs = 5
iter_n = 0
for e in range(epochs):
    print('Epoch {}'.format(e+1))
    running_loss = 0
    for i, (images,labels) in enumerate(trainloader):
        images = images.requires_grad_()
        optimizer.zero_grad()
        logits = model.forward(images)
        loss = criterion(logits,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        iter_n += 1
        if iter_n%500 == 0:
            correct = 0
            total = 0
            with torch.no_grad():
                for images, labels in testloader:
                    logit = model(images)
                    _, predicted = torch.max(logit.data, 1)
                    total += labels.size(0)

                    # Total correct predictions
                    correct += (predicted == labels).sum()

                accuracy = 100 * correct / total

                # Print Loss
                print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter_n, loss.item(), accuracy))
    else:
        accuracy = 0
        with torch.no_grad():
            for images,labels in testloader:
                logit = model(images)
                ps = logit #torch.exp(logit)
                top_p,top_class = ps.topk(1,dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))                
        print('Train accuracy: {}'.format(running_loss))
        print('Test accuracy: {}'.format((accuracy/len(testloader)).data.numpy()))

Epoch 1


/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Iteration: 500. Loss: 0.26290783286094666. Accuracy: 91
Train accuracy: 451.24602250754833
Test accuracy: 0.9334676265716553
Epoch 2
Iteration: 1000. Loss: 0.18996667861938477. Accuracy: 94
Train accuracy: 115.83999397605658
Test accuracy: 0.9522674679756165
Epoch 3
Iteration: 1500. Loss: 0.25625938177108765. Accuracy: 95
Train accuracy: 81.95738133788109
Test accuracy: 0.9662501215934753
Epoch 4
Iteration: 2000. Loss: 0.05149219185113907. Accuracy: 96
Train accuracy: 65.95421887934208
Test accuracy: 0.9700831174850464
Epoch 5
Iteration: 2500. Loss: 0.1725318729877472. Accuracy: 97
Iteration: 3000. Loss: 0.0761098563671112. Accuracy: 97
Train accuracy: 56.789038985967636
Test accuracy: 0.9727830290794373
